# nn.py

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




# Initialization and Environment Setup
def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def load_test_data(data1_test_path_A, data2_test_path_A, df_Y_test_path_A,
                    data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                      data1_test_path_C, data2_test_path_C, df_Y_test_path_C):
    
    data1_test_A = pd.read_csv(data1_test_path_A)
    data2_test_A = pd.read_csv(data2_test_path_A)
    df_Y_test_A = pd.read_csv(df_Y_test_path_A)
    data1_test_B = pd.read_csv(data1_test_path_B)
    data2_test_B = pd.read_csv(data2_test_path_B)
    df_Y_test_B = pd.read_csv(df_Y_test_path_B)
    data1_test_C = pd.read_csv(data1_test_path_C)
    data2_test_C = pd.read_csv(data2_test_path_C)
    df_Y_test_C = pd.read_csv(df_Y_test_path_C)
    print('Test data loaded successfully')
    return data1_test_A, data2_test_A, df_Y_test_A, data1_test_B, data2_test_B, df_Y_test_B, data1_test_C, data2_test_C, df_Y_test_C



def cartesian_product(data1, data2):
    data1 = data1.iloc[:, 1:]
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        combined_data = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield combined_data, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model

# Evaluation Function
def evaluate_model_on_test_data(model, data1_test, data2_test, df_Y_test, epoch, loss_fn, device, test_batch_size=128):
    model.eval()
    avg_test_losses = []
    mae_values = []
    rmse_values = []
    r2_values = []
    correlation_coefficients = []
    p_values = []
    gene_batches = 1
    for batch_X, batch_Y in cartesian_product_generator(data1_test, data2_test, df_Y_test, gene_batches):
        X_test = torch.tensor(batch_X.values, dtype=torch.float32).to(device)
        Y_test = torch.tensor(batch_Y.values.reshape(-1, 1), dtype=torch.float32).to(device)
        test_data = TensorDataset(X_test, Y_test)
        test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

        test_loss = 0.0
        actual_outputs = []
        predicted_outputs = []
        with torch.no_grad():
            for inputs, targets in test_dataloader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                test_loss += loss.item()
                actual_outputs.extend(targets.cpu().numpy().flatten().tolist())
                predicted_outputs.extend(outputs.cpu().numpy().flatten().tolist())

        avg_test_loss = test_loss / len(test_dataloader)
        avg_test_losses.append(avg_test_loss)
        correlation_coefficient, p_value = pearsonr(actual_outputs, predicted_outputs)
        correlation_coefficients.append(correlation_coefficient)
        p_values.append(p_value)
        mae = mean_absolute_error(actual_outputs, predicted_outputs)
        rmse = np.sqrt(mean_squared_error(actual_outputs, predicted_outputs))
        r2 = r2_score(actual_outputs, predicted_outputs)
        mae_values.append(mae)
        rmse_values.append(rmse)
        r2_values.append(r2)

    epochs = []
    for minor in range(1, len(data1_test) + 1):
        epoched = f'{epoch}.{minor:02}'  
        epochs.append(float(epoched))

    metrics_df = pd.DataFrame({
        'Epoch': epochs,
        'Correlation_Coefficient': correlation_coefficients,
        'P_Value': p_values,
        'Test_Loss': avg_test_losses,
        'MAE': mae_values,
        'RMSE': rmse_values,
        'R2_Score': r2_values
    })
    return metrics_df

# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)

        
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs
    
    for epoch in range(start_epoch, end_epoch):
        model.train()
        counter = 0
        model = load_model(model, epoch, model_save_path)
        for batch_X, batch_Y in cartesian_product_generator(data1, data2, df_Y, batch_size1):
            X_train = torch.tensor(batch_X.values, dtype=torch.float32).to(device)
            Y_train = torch.tensor(batch_Y.values.reshape(-1, 1), dtype=torch.float32).to(device)
            train_data = TensorDataset(X_train, Y_train)
            train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)

            train_loss = 0.0
            for inputs, targets in train_dataloader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            counter += 1
            print(f"Epoch {epoch}.{counter}: Avg. training loss = {train_loss / len(train_dataloader):.4f}")

        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        
        # Evaluate the model on test data after each epoch

        data1_test_A, data2_test_A, df_Y_test_A, data1_test_B, data2_test_B, df_Y_test_B, data1_test_C, data2_test_C, df_Y_test_C = load_test_data(data1_test_path_A, data2_test_path_A, df_Y_test_path_A,
                                                                                                                                                data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                                                                                                                                                  data1_test_path_C, data2_test_path_C, df_Y_test_path_C)



        metrics_df_A = evaluate_model_on_test_data(model, data1_test_A, data2_test_A, df_Y_test_A, epoch, loss_fn, device, test_batch_size)
        metrics_path_A = os.path.join(model_save_path, f"A_metrics_epoch_{epoch}.csv")
        metrics_df_A.to_csv(metrics_path_A, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_A}')

        metrics_df_B = evaluate_model_on_test_data(model, data1_test_B, data2_test_B, df_Y_test_B, epoch, loss_fn, device, test_batch_size)
        metrics_path_B = os.path.join(model_save_path, f"B_metrics_epoch_{epoch}.csv")
        metrics_df_B.to_csv(metrics_path_B, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_B}')

        metrics_df_C = evaluate_model_on_test_data(model, data1_test_C, data2_test_C, df_Y_test_C, epoch, loss_fn, device, test_batch_size)
        metrics_path_C = os.path.join(model_save_path, f"C_metrics_epoch_{epoch}.csv")
        metrics_df_C.to_csv(metrics_path_C, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_C}')

main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=500,learning_rate=0.001,num_epochs=20,
                   model_save_path='/mnt/data/macaulay/model_state2/',test_batch_size=128)

    

## OPTIMIZATION AND BATCHING

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




# Initialization and Environment Setup
def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def load_test_data(data1_test_path_A, data2_test_path_A, df_Y_test_path_A,
                    data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                      data1_test_path_C, data2_test_path_C, df_Y_test_path_C):
    
    data1_test_A = pd.read_csv(data1_test_path_A)
    data2_test_A = pd.read_csv(data2_test_path_A)
    df_Y_test_A = pd.read_csv(df_Y_test_path_A)
    data1_test_B = pd.read_csv(data1_test_path_B)
    data2_test_B = pd.read_csv(data2_test_path_B)
    df_Y_test_B = pd.read_csv(df_Y_test_path_B)
    data1_test_C = pd.read_csv(data1_test_path_C)
    data2_test_C = pd.read_csv(data2_test_path_C)
    df_Y_test_C = pd.read_csv(df_Y_test_path_C)
    print('Test data loaded successfully')
    return data1_test_A, data2_test_A, df_Y_test_A, data1_test_B, data2_test_B, df_Y_test_B, data1_test_C, data2_test_C, df_Y_test_C



def cartesian_product(data1, data2):
    data1 = data1.iloc[:, 1:]
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        combined_data = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield combined_data, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model

# Evaluation Function
def evaluate_model_on_test_data(model, data1_test, data2_test, df_Y_test, epoch, loss_fn, device, test_batch_size=128):
    model.eval()
    avg_test_losses = []
    mae_values = []
    rmse_values = []
    r2_values = []
    correlation_coefficients = []
    p_values = []
    gene_batches = 1
    for batch_X, batch_Y in cartesian_product_generator(data1_test, data2_test, df_Y_test, gene_batches):
        X_test = torch.tensor(batch_X.values, dtype=torch.float32).to(device)
        Y_test = torch.tensor(batch_Y.values.reshape(-1, 1), dtype=torch.float32).to(device)
        test_data = TensorDataset(X_test, Y_test)
        test_dataloader = DataLoader(test_data, batch_size=test_batch_size, shuffle=True)

        test_loss = 0.0
        actual_outputs = []
        predicted_outputs = []
        with torch.no_grad():
            for inputs, targets in test_dataloader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                test_loss += loss.item()
                actual_outputs.extend(targets.cpu().numpy().flatten().tolist())
                predicted_outputs.extend(outputs.cpu().numpy().flatten().tolist())

        avg_test_loss = test_loss / len(test_dataloader)
        avg_test_losses.append(avg_test_loss)
        correlation_coefficient, p_value = pearsonr(actual_outputs, predicted_outputs)
        correlation_coefficients.append(correlation_coefficient)
        p_values.append(p_value)
        mae = mean_absolute_error(actual_outputs, predicted_outputs)
        rmse = np.sqrt(mean_squared_error(actual_outputs, predicted_outputs))
        r2 = r2_score(actual_outputs, predicted_outputs)
        mae_values.append(mae)
        rmse_values.append(rmse)
        r2_values.append(r2)

    epochs = []
    for minor in range(1, len(data1_test) + 1):
        epoched = f'{epoch}.{minor:02}'  
        epochs.append(float(epoched))

    metrics_df = pd.DataFrame({
        'Epoch': epochs,
        'Correlation_Coefficient': correlation_coefficients,
        'P_Value': p_values,
        'Test_Loss': avg_test_losses,
        'MAE': mae_values,
        'RMSE': rmse_values,
        'R2_Score': r2_values
    })
    return metrics_df

# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)

        
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs
    
    for epoch in range(start_epoch, end_epoch):
        model.train()
        counter = 0
        model = load_model(model, epoch, model_save_path)
        for batch_X, batch_Y in cartesian_product_generator(data1, data2, df_Y, batch_size1):
            X_train = torch.tensor(batch_X.values, dtype=torch.float32).to(device)
            Y_train = torch.tensor(batch_Y.values.reshape(-1, 1), dtype=torch.float32).to(device)
            train_data = TensorDataset(X_train, Y_train)
            train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)

            train_loss = 0.0
            for inputs, targets in train_dataloader:
                inputs, targets = inputs.to(device), targets.to(device)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = loss_fn(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
            counter += 1
            print(f"Epoch {epoch}.{counter}: Avg. training loss = {train_loss / len(train_dataloader):.4f}")

        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        
        # Evaluate the model on test data after each epoch

        data1_test_A, data2_test_A, df_Y_test_A, data1_test_B, data2_test_B, df_Y_test_B, data1_test_C, data2_test_C, df_Y_test_C = load_test_data(data1_test_path_A, data2_test_path_A, df_Y_test_path_A,
                                                                                                                                                data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                                                                                                                                                  data1_test_path_C, data2_test_path_C, df_Y_test_path_C)



        metrics_df_A = evaluate_model_on_test_data(model, data1_test_A, data2_test_A, df_Y_test_A, epoch, loss_fn, device, test_batch_size)
        metrics_path_A = os.path.join(model_save_path, f"A_metrics_epoch_{epoch}.csv")
        metrics_df_A.to_csv(metrics_path_A, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_A}')

        metrics_df_B = evaluate_model_on_test_data(model, data1_test_B, data2_test_B, df_Y_test_B, epoch, loss_fn, device, test_batch_size)
        metrics_path_B = os.path.join(model_save_path, f"B_metrics_epoch_{epoch}.csv")
        metrics_df_B.to_csv(metrics_path_B, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_B}')

        metrics_df_C = evaluate_model_on_test_data(model, data1_test_C, data2_test_C, df_Y_test_C, epoch, loss_fn, device, test_batch_size)
        metrics_path_C = os.path.join(model_save_path, f"C_metrics_epoch_{epoch}.csv")
        metrics_df_C.to_csv(metrics_path_C, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_C}')

main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=500,learning_rate=0.001,num_epochs=20,
                   model_save_path='/mnt/data/macaulay/model_state2/',test_batch_size=128)

    

In [4]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




# Initialization and Environment Setup
def initialize_environment(data1_path, data2_path, df_Y_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')

    return device, data1, data2, df_Y



def main_training_loop(data1_path, data2_path, df_Y_path, num_epochs, model_save_path):
    

    data1, data2, df_Y = initialize_environment(data1_path, data2_path, df_Y_path)
    display(data1)
    display(data2)
    display(df_Y)

        
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs



main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   num_epochs=20,
                   model_save_path='/mnt/data/macaulay/model_state2/')

   

Device: cuda
Number of available GPUs: 1


Training data loaded successfully
Model initialized successfully


ValueError: too many values to unpack (expected 3)

## Training

In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model


# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)
    # data1 = data1.iloc[:300]

    # display(data1)
    # display(data2)
    # display(df_Y)



    # import time

    # start_epoch = 1
    # end_epoch = 20
    from sklearn.model_selection import train_test_split
    from scipy.stats import pearsonr
    # print(len(data1))
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs

    for epoch in range(start_epoch, end_epoch):
        max_index = (len(data1) // batch_size1) * batch_size1
        # print(max_index)
        # # time.sleep(10000)

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []

        for j in range(0, max_index, batch_size1):
            batched_data1 = data1.iloc[j:j+batch_size1]
            model.train()
            model = load_model(model, epoch, model_save_path)
            train_correlations = []
            train_lossess = []
            test_correlations = []
            test_lossess = []
            for batch_X, batch_Y in cartesian_product_generator(batched_data1, data2, df_Y, batch_size1):
                
                # Concatenate X and Y DataFrames along the columns axis
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)            

                # Shuffle data
                combined = combined.sample(frac=1).reset_index(drop=True)
                # display(shuffled_combined)
                # print('shuffled')
                # time.sleep(1000)

                # Split shuffled data back into X and Y
                shuffled_X = combined.iloc[:, :-batch_Y.shape[1]]
                shuffled_Y = combined.iloc[:, -batch_Y.shape[1]:]

                # Then use train_test_split to split the shuffled data into training and test sets
                X_train, X_test, Y_train, Y_test = train_test_split(shuffled_X, shuffled_Y, test_size=0.1)
                X_train = X_train.drop(columns=["Gene"])
                X_test = X_test.drop(columns=["Gene"])

                # print(X_train.select_dtypes(include=['object']).head())
                # display(X_test)
                # display(Y_train)
                # display(Y_test)
                # Convert training data to tensor and create dataloader
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations.append(train_correlation)
                    train_lossess.append(loss.item())

                # Evaluate on test data
                model.eval()
                with torch.no_grad():
                    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    # Convert tensors to numpy arrays for correlation computation
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    test_correlations.append(correlation)
                    test_lossess.append(test_loss.item())
            
            mean_train_correlation = np.mean(train_correlations)
            mean_train_loss = np.mean(train_lossess)
            mean_test_correlation = np.mean(test_correlations)
            mean_test_loss = np.mean(test_lossess)

            epoch_train_correlations.append(mean_train_correlation)
            epoch_train_losses.append(mean_train_loss)
            epoch_test_correlations.append(mean_test_correlation)
            epoch_test_losses.append(mean_test_loss)



            print(f"Epoch {epoch}.{j}: Avg. training loss = {mean_train_loss:.4f}, Training Pearson correlation = {mean_train_correlation:.4f}")
            print(f"Epoch {epoch}.{j}: Test loss = {mean_test_loss:.4f}, Pearson correlation = {mean_test_correlation:.4f}")

        print(f"Epoch {epoch}: Avg. training loss = {np.mean(epoch_train_losses):.4f}, Training Pearson correlation = {np.mean(epoch_train_correlations):.4f}")
        print(f"Epoch {epoch}: Test loss = {np.mean(epoch_test_losses):.4f}, Pearson correlation = {np.mean(epoch_test_correlations):.4f}")

        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
            if epoch == 1:
                f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
            f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')




main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=500,learning_rate=0.001,num_epochs=10,
                   model_save_path='/mnt/data/macaulay/datas/genepedia/',test_batch_size=128)

   

Device: cuda
Number of available GPUs: 1
Training data loaded successfully
Model initialized successfully
300
300
No saved model found. Training from scratch.
97100
1
Epoch 1.0: Avg. training loss = 0.1570, Training Pearson correlation = 0.3173
Epoch 1.0: Test loss = 0.0829, Pearson correlation = 0.6449
No saved model found. Training from scratch.
97100
1
Epoch 1.100: Avg. training loss = 0.1385, Training Pearson correlation = 0.2063
Epoch 1.100: Test loss = 0.1055, Pearson correlation = 0.4553
No saved model found. Training from scratch.
97100
1
Epoch 1.200: Avg. training loss = 0.1419, Training Pearson correlation = 0.0836
Epoch 1.200: Test loss = 0.1219, Pearson correlation = 0.2599
Epoch 1: Avg. training loss = 0.1458, Training Pearson correlation = 0.2024
Epoch 1: Test loss = 0.1034, Pearson correlation = 0.4533
300
Model 1 loaded successfully for epoch 2
97100
1
Epoch 2.0: Avg. training loss = 0.0948, Training Pearson correlation = 0.4904
Epoch 2.0: Test loss = 0.0715, Pearson co

KeyboardInterrupt: 

## GROUP A VAL SET

In [1]:
# #a.py
# """

# Goal
# Test the model on the saved models, and calculate the metrics for each gene concatinated with the cell line embeddings

# it calculate the correlation by each seen gene
# """
# import torch 
# from torch import nn
# from torch.utils.data import DataLoader, TensorDataset
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.stats import pearsonr
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import os
# import torch.nn.functional as F
# from functions import FullConnectedBlock, NeuralNetwork
# pd.options.mode.chained_assignment = None





# # Initialize lists to store metrics


# avg_test_losses = []
# mae_values = []
# rmse_values = []
# r2_values = []
# correlation_coefficients = []
# p_values = []



# # Loop through the saved models
# # for epoch in range(num_epochs):

# target_epochs = [2]

# # Loop through the target epochs
# for epoch in target_epochs:
#     #model_epoch = epoch
#     Y_data_count = 0

#     model_path = f'/mnt/data/macaulay/datas/genepedia/crispr_fc1_model_state_epoch_{epoch}.pth'
#     if os.path.exists(model_path):
        

#         data1_test = pd.read_csv("/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv") #30 >>
#         data2_test = pd.read_csv("/mnt/data/macaulay/datas/training_gene_embeddings.csv") #17000 >>
#         df_Y_test = pd.read_csv('/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv') #>>
#         print('Test data loaded successfully')




#         batch_size1 = len(data1_test)
#         batch_size_Y = len(data1_test)
#         batch_size2 = 1
#         combined_batches_test = []

#         input_dim = data1_test.shape[1] + data2_test.shape[1] - 1
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#         print('Device:', device)
#         model = NeuralNetwork(input_dim)

#         # If multiple GPUs
#         if torch.cuda.device_count() >= 4:
#             print("Using 4 GPUs!")
#             model = nn.DataParallel(model, device_ids=list(range(4)))

#         model.to(device)
#         model.load_state_dict(torch.load(model_path))
#         model.eval()  # Set the model to evaluation mode

#         # Define the loss function as Mean Squared Error
#         loss_fn = nn.MSELoss()

        

#         for i in range(0, len(data2_test), batch_size2):  # Loop through data2_test
#             batch_data2_test = data2_test.iloc[i:i+batch_size2]
#             for j in range(0, len(data1_test), batch_size1):  # Loop through data1_test
#                 batch_data1_test = data1_test.iloc[j:j+batch_size1]
#                 batch_data1_test['key'] = 1
#                 batch_data2_test['key'] = 1
#                 batch_data1_test = batch_data1_test[list(data1_test.columns[0:]) + ['key']]
#                 batch_data2_test = batch_data2_test[list(data2_test.columns[1:]) + ['key']]
#                 combined_batch_test = pd.merge(batch_data1_test, batch_data2_test, on='key').drop(columns=['key'])
#                 combined_batches_test.append(combined_batch_test)

#                 X_test = pd.concat(combined_batches_test)
#                 combined_batches_test = []
#                 Y_test = df_Y_test.iloc[Y_data_count:(batch_size_Y * (i+1))]
#                 Y_data_count += batch_size_Y

                
#                 X_test1 = torch.tensor(X_test.values, dtype=torch.float32)
#                 Y_test1 = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32)
#                 test_data = TensorDataset(X_test1, Y_test1)
#                 test_dataloader = DataLoader(test_data, batch_size=128, shuffle=True)

#                 #print('Test data preprocessed successfully')
#                 ##################### The first 1 Batch which forms 100 rows after concat, are preprocessed and fed to the neural network under the same loop #####################

#                 # Evaluate the model on the test data
#                 test_loss = 0.0
#                 actual_outputs = []
#                 predicted_outputs = []
#                 with torch.no_grad():  # Disable gradient calculation
#                     for inputs, targets in test_dataloader:
#                         inputs = inputs.to(device)
#                         targets = targets.to(device)
#                         outputs = model(inputs)
#                         loss = loss_fn(outputs, targets)
#                         test_loss += loss.item()
#                         actual_outputs.extend(targets.cpu().numpy().flatten().tolist())
#                         predicted_outputs.extend(outputs.cpu().numpy().flatten().tolist())

#                 avg_test_loss = test_loss / len(test_dataloader)
#                 avg_test_losses.append(avg_test_loss)

#                 # Calculate Pearson correlation coefficient
#                 correlation_coefficient, p_value = pearsonr(actual_outputs, predicted_outputs)
#                 correlation_coefficients.append(correlation_coefficient)
#                 p_values.append(p_value)  # Append the p-value

#                 mae = mean_absolute_error(actual_outputs, predicted_outputs)
#                 rmse = np.sqrt(mean_squared_error(actual_outputs, predicted_outputs))
#                 r2 = r2_score(actual_outputs, predicted_outputs)

#                 # Append metrics to their respective lists
#                 mae_values.append(mae)
#                 rmse_values.append(rmse)
#                 r2_values.append(r2)

#                 print(f'Epoch {epoch} : gene {i+1}/{target_epochs}: Avg. test loss = {avg_test_loss:.4f}')
#                 print(f'Epoch {epoch} : gene {i+1}/{target_epochs}: Correlation Coefficient = {correlation_coefficient:.4f}')
#                 print(f'Epoch {epoch} : gene {i+1}/{target_epochs}: Mean Absolute Error = {mae:.4f}')
#                 print(f'Epoch {epoch} : gene {i+1}/{target_epochs}: Root Mean Square Error = {rmse:.4f}')
#                 print(f'Epoch {epoch} : gene {i+1}/{target_epochs}: R2 Score = {r2:.4f}')


#     else:
#         print(f'Model for epoch {epoch} not found!')


# # Create a list to store the full epoch labels
# epochs = []

# # Loop through the target epochs
# for major in target_epochs:
#     for minor in range(1, len(data2_test) + 1):
#         epoch = f'{major}.{minor:02}'  # Format as a string with two decimal places
#         epochs.append(float(epoch))

# # Create a DataFrame to hold the metrics
# metrics_df = pd.DataFrame({
#     'Epoch': epochs,
#     'Correlation_Coefficient': correlation_coefficients,
#     'P_Value': p_values,  # Include the p-values
#     'Test_Loss': avg_test_losses,
#     'MAE': mae_values,
#     'RMSE': rmse_values,
#     'R2_Score': r2_values
# })

# # Write the DataFrame to a CSV file
# metrics_path = f'/mnt/data/macaulay/datas/genepedia/A_seen_genewise_correlation_metrics_summary_crispr_{target_epochs[0]}.csv'
# metrics_df.to_csv(metrics_path, index=False)
# print(f'Metrics saved to {metrics_path}')

























import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr



def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model


def load_test_data(data1_test_path_A, data2_test_path_A, df_Y_test_path_A,
                    data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                      data1_test_path_C, data2_test_path_C, df_Y_test_path_C):
    
    data1_test_A = pd.read_csv(data1_test_path_A)
    data2_test_A = pd.read_csv(data2_test_path_A)
    df_Y_test_A = pd.read_csv(df_Y_test_path_A)
    data1_test_B = pd.read_csv(data1_test_path_B)
    data2_test_B = pd.read_csv(data2_test_path_B)
    df_Y_test_B = pd.read_csv(df_Y_test_path_B)
    data1_test_C = pd.read_csv(data1_test_path_C)
    data2_test_C = pd.read_csv(data2_test_path_C)
    df_Y_test_C = pd.read_csv(df_Y_test_path_C)
    print('Test data loaded successfully')
    return data1_test_A, data2_test_A, df_Y_test_A, data1_test_B, data2_test_B, df_Y_test_B, data1_test_C, data2_test_C, df_Y_test_C


# Evaluation Function
def evaluate_model_on_test_data(model, data1_test, data2_test, df_Y_test, epoch, loss_fn, device, test_batch_size=128):
    model.eval()
    # print('data1_test')
    # display(data1_test)
    # print('data2_test')
    # display(data2_test)
    # print('df_Y_test')
    # display(df_Y_test)

    



    avg_test_losses = []
    mae_values = []
    rmse_values = []
    r2_values = []
    correlation_coefficients = []
    p_values = []
    gene_batches = 1
    counter = 0
    for batch_X, batch_Y in cartesian_product_generator(data1_test, data2_test, df_Y_test, gene_batches):
        
        batch_X = batch_X.drop(columns=["Gene"])

        
        # print('batch_X')
        # display(batch_X)
        # print('batch_Y')
        # display(batch_Y)
        

        
        X_test = torch.tensor(batch_X.values, dtype=torch.float32).to(device)
        Y_test = torch.tensor(batch_Y.values.reshape(-1, 1), dtype=torch.float32).to(device)
        predictions = model(X_test)
        test_loss = loss_fn(predictions, Y_test)
        predictions_np = predictions.cpu().detach().numpy()

        Y_test_np = Y_test.cpu().numpy()
        
        correlation, pval = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
        
        correlation_coefficients.append(correlation)
        avg_test_losses.append(test_loss.item())
        p_values.append(pval)
        counter += 1
        print(f'Epoch {epoch} gene {counter} : Avg. test loss = {test_loss.item():.4f}, correlation = {correlation:.4f}, p-value = {pval:.4f}')



    epochs = []
    for minor in range(1, len(data1_test) + 1):
        epoched = f'{epoch}.{minor:02}'  
        epochs.append(float(epoched))

    metrics_df = pd.DataFrame({
        'Epoch': epochs,
        'Correlation_Coefficient': correlation_coefficients,
        'P_Value': p_values,
        'Test_Loss': avg_test_losses,
        # 'MAE': mae_values,
        # 'RMSE': rmse_values,
        # 'R2_Score': r2_values
    })
    return metrics_df



# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    
# Evaluate the model on test data after each epoch
    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)

    data1_test_A, data2_test_A, df_Y_test_A, data1_test_B, data2_test_B, df_Y_test_B, data1_test_C, data2_test_C, df_Y_test_C = load_test_data(data1_test_path_A, data2_test_path_A, df_Y_test_path_A,
                                                                                                                                                data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                                                                                                                                                  data1_test_path_C, data2_test_path_C, df_Y_test_path_C)
    target_epochs = [20]

    
    for epoch in target_epochs:


        metrics_df_A = evaluate_model_on_test_data(model, data1_test_A, data2_test_A, df_Y_test_A, epoch, loss_fn, device, test_batch_size)
        metrics_path_A = os.path.join(model_save_path, f"A_metrics_epoch_{epoch}.csv")
        metrics_df_A.to_csv(metrics_path_A, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_A}')

        metrics_df_B = evaluate_model_on_test_data(model, data1_test_B, data2_test_B, df_Y_test_B, epoch, loss_fn, device, test_batch_size)
        metrics_path_B = os.path.join(model_save_path, f"B_metrics_epoch_{epoch}.csv")
        metrics_df_B.to_csv(metrics_path_B, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_B}')

        metrics_df_C = evaluate_model_on_test_data(model, data1_test_C, data2_test_C, df_Y_test_C, epoch, loss_fn, device, test_batch_size)
        metrics_path_C = os.path.join(model_save_path, f"C_metrics_epoch_{epoch}.csv")
        metrics_df_C.to_csv(metrics_path_C, index=False)
        print(f'Metrics for epoch {epoch} saved to {metrics_path_C}')







main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=500,learning_rate=0.001,num_epochs=10,
                   model_save_path='/mnt/data/macaulay/datas/genepedia/',test_batch_size=128)

   


Device: cuda
Number of available GPUs: 1
Training data loaded successfully
Model initialized successfully
Test data loaded successfully
30
Epoch 20 gene 1 : Avg. test loss = 0.2071, correlation = -0.2081, p-value = 0.2699
30
Epoch 20 gene 2 : Avg. test loss = 0.0287, correlation = 0.0570, p-value = 0.7650
30
Epoch 20 gene 3 : Avg. test loss = 0.1351, correlation = -0.2641, p-value = 0.1584
30
Epoch 20 gene 4 : Avg. test loss = 0.0171, correlation = -0.0503, p-value = 0.7918
30
Epoch 20 gene 5 : Avg. test loss = 0.0551, correlation = 0.0297, p-value = 0.8764
30
Epoch 20 gene 6 : Avg. test loss = 0.0659, correlation = 0.2530, p-value = 0.1773
30
Epoch 20 gene 7 : Avg. test loss = 0.0236, correlation = -0.1901, p-value = 0.3143
30
Epoch 20 gene 8 : Avg. test loss = 0.0673, correlation = -0.0127, p-value = 0.9468
30
Epoch 20 gene 9 : Avg. test loss = 0.0475, correlation = -0.4006, p-value = 0.0282
30
Epoch 20 gene 10 : Avg. test loss = 0.0420, correlation = 0.2018, p-value = 0.2848
30
Epoc

: 

In [ ]:
scp -r macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/training_gene_embeddings.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/training_crispr.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/test_gene_embeddings.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/test_gene_embeddings.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv macaulay@sahu.cs.unm.edu:/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv asahu@kraken.dfci.harvard.edu:/homes/asahu/liulab_home/macaulay/code/datas/

In [2]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    m_counter = 0
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        if m_counter == 0:
            print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
            m_counter += 1
    else:
        if m_counter == 0:
            print('No saved model found. Training from scratch.')
            m_counter += 1
    return model


# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)
    # data1 = data1.iloc[:300]

    # display(data1)
    # display(data2)
    # display(df_Y)



    # import time

    # start_epoch = 1
    # end_epoch = 20
    from sklearn.model_selection import train_test_split
    from scipy.stats import pearsonr
    # print(len(data1))
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs

    for epoch in range(start_epoch, end_epoch):
        max_index = (len(data1) // batch_size1) * batch_size1
        # print(max_index)
        # # time.sleep(10000)

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []

        for j in range(0, max_index, batch_size1):
            batched_data1 = data1.iloc[j:j+batch_size1]
            model.train()
            model = load_model(model, epoch, model_save_path)
            train_correlations = []
            train_lossess = []
            test_correlations = []
            test_lossess = []
            for batch_X, batch_Y in cartesian_product_generator(batched_data1, data2, df_Y, batch_size1):
                
                # Concatenate X and Y DataFrames along the columns axis
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)            

                # Shuffle data
                #combined = combined.sample(frac=1).reset_index(drop=True)
                # display(shuffled_combined)
                # print('shuffled')
                # time.sleep(1000)

                # Split shuffled data back into X and Y
                shuffled_X = combined.iloc[:, :-batch_Y.shape[1]]
                shuffled_Y = combined.iloc[:, -batch_Y.shape[1]:]

                # Then use train_test_split to split the shuffled data into training and test sets
                X_train, X_test, Y_train, Y_test = train_test_split(shuffled_X, shuffled_Y, test_size=0.1, shuffle=False)
                # display(X_test)
                # display(Y_test)

                # Copy gene name
                batch_gene_name = X_test['Gene'].iloc[0]
                print(batch_gene_name)
                X_train = X_train.drop(columns=["Gene"])
                X_test = X_test.drop(columns=["Gene"])

                # print(X_train.select_dtypes(include=['object']).head())
                # display(X_test)
                # # display(Y_train)
                # display(Y_test)
                # import time
                
                # Convert training data to tensor and create dataloader
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations.append(train_correlation)
                    train_lossess.append(loss.item())

                # Evaluate on test data
                model.eval()
                with torch.no_grad():
                    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    # Convert tensors to numpy arrays for correlation computation
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    test_correlations.append(correlation)
                    test_lossess.append(test_loss.item())
            
            mean_train_correlation = np.mean(train_correlations)
            mean_train_loss = np.mean(train_lossess)
            mean_test_correlation = np.mean(test_correlations)
            mean_test_loss = np.mean(test_lossess)

            epoch_train_correlations.append(mean_train_correlation)
            epoch_train_losses.append(mean_train_loss)
            epoch_test_correlations.append(mean_test_correlation)
            epoch_test_losses.append(mean_test_loss)

            if j//2 == 0:

                print(f"Epoch {epoch}.{j}: Training Pearson correlation = {mean_train_correlation:.4f}")
                print(f"Epoch {epoch}.{j}.{batch_gene_name}: Pearson correlation = {mean_test_correlation:.4f}")

            with open(os.path.join(model_save_path, f'metrics_by_gene.txt'), 'a') as f:
                if epoch == 1:
                    f.write('Epoch,Gene,gene_correlation\n')
                f.write(f'Epoch{epoch},{batch_gene_name},{mean_test_correlation:.4f}\n')
              


        print(f"Epoch {epoch}: Avg. training loss = {np.mean(epoch_train_losses):.4f}, Training Pearson correlation = {np.mean(epoch_train_correlations):.4f}")
        print(f"Epoch {epoch}: Test loss = {np.mean(epoch_test_losses):.4f}, Pearson correlation = {np.mean(epoch_test_correlations):.4f}")

        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
            if epoch == 1:
                f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
            f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')




main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=1,learning_rate=0.001,num_epochs=10,
                   model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

   


Device: cuda
Number of available GPUs: 1


KeyboardInterrupt: 

In [3]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model


# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)
    # data1 = data1.iloc[:300]

    # display(data1)
    # display(data2)
    # display(df_Y)



    # import time

    # start_epoch = 1
    # end_epoch = 20
    from sklearn.model_selection import train_test_split
    from scipy.stats import pearsonr
    # print(len(data1))
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs


\

    gene_predictions = {}
    gene_actuals = {}


    for epoch in range(start_epoch, end_epoch):
        max_index = (len(data1) // batch_size1) * batch_size1
        # print(max_index)
        # # time.sleep(10000)

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []

        for j in range(0, max_index, batch_size1):
            batched_data1 = data1.iloc[j:j+batch_size1]
            model.train()
            model = load_model(model, epoch, model_save_path)
            train_correlations = []
            train_lossess = []
            test_correlations = []
            test_lossess = []
            for batch_X, batch_Y in cartesian_product_generator(batched_data1, data2, df_Y, batch_size1):
                
                # Concatenate X and Y DataFrames along the columns axis
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)            

                
                # Split  data back into X and Y
                datas_X = combined.iloc[:, :-batch_Y.shape[1]]
                datas_Y = combined.iloc[:, -batch_Y.shape[1]:]

                X_train = datas_X.iloc[:int(len(datas_X)-9710)]
                X_test = datas_X.iloc[int(len(datas_X)-9710):]
                Y_train = datas_Y.iloc[:int(len(datas_Y)-9710)]
                Y_test = datas_Y.iloc[int(len(datas_Y)-9710):]

                # Then use train_test_split to split the shuffled data into training and test sets
                X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.1, shuffle=False)
                X_train = X_train.drop(columns=["Gene"])
                

                # print(X_train.select_dtypes(include=['object']).head())
                # display(X_test)
                # display(Y_train)
                # display(Y_test)
                # Convert training data to tensor and create dataloader
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations.append(train_correlation)
                    train_lossess.append(loss.item())

                # Evaluate on test data

                model.eval()
                with torch.no_grad():
                    # Convert test data to tensor and evaluate
                    X_test_tensor = torch.tensor(X_test.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    # Convert tensors to numpy arrays for correlation computation
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    epoch_test_correlations.append(correlation)
                    epoch_test_losses.append(test_loss.item())

                    # Aggregate predictions and actual values for each gene
                    for gene in X_test['Gene'].unique():
                        gene_mask = (X_test['Gene'] == gene)
                        X_test_gene = X_test[gene_mask]
                        Y_test_gene = Y_test[gene_mask]

                        X_test_tensor = torch.tensor(X_test_gene.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                        Y_test_tensor = torch.tensor(Y_test_gene.values.reshape(-1, 1), dtype=torch.float32).to(device)

                        predictions_gene = model(X_test_tensor)

                        predictions_gene_np = predictions_gene.cpu().numpy().squeeze()
                        Y_test_gene_np = Y_test_tensor.cpu().numpy().squeeze()
                        gene_predictions[gene] = gene_predictions.get(gene, []) + predictions_gene_np.tolist()
                        gene_actuals[gene] = gene_actuals.get(gene, []) + Y_test_gene_np.tolist()

            # Calculate Pearson correlation for each gene
            gene_correlations = {}
            for gene in gene_predictions.keys():
                correlation, _ = pearsonr(gene_predictions[gene], gene_actuals[gene])
                gene_correlations[gene] = correlation

            # Print or save gene correlations
            for gene, corr in gene_correlations.items():
                print(f"Gene {gene}: Pearson Correlation = {corr:.4f}")
                with open(os.path.join(model_save_path, f'gene_correlations.txt'), 'a') as f:
                    if epoch == 1:
                        f.write('Epoch,gene,correlation\n')
                    f.write(f'Epoch{epoch},{gene},{corr:.4f}\n')
                   

            # Save model state and metrics
            torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
            with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
                if epoch == 1:
                    f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
                f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')





main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=500,learning_rate=0.001,num_epochs=10,
                   model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

   


Device: cuda
Number of available GPUs: 1


Training data loaded successfully
Model initialized successfully
Model 11 loaded successfully for epoch 12
485500
Gene PHF23: Pearson Correlation = 0.0500
Gene CDH10: Pearson Correlation = 0.0192
Gene INPP4A: Pearson Correlation = -0.0673
Gene RAB27B: Pearson Correlation = 0.0604
Gene PSMA4: Pearson Correlation = -0.0066
Gene MYO16: Pearson Correlation = -0.0100
Gene LSG1: Pearson Correlation = 0.0955
Gene PARP3: Pearson Correlation = 0.0140
Gene TNC: Pearson Correlation = -0.0658
Gene THAP3: Pearson Correlation = 0.0710
Gene RIPOR3: Pearson Correlation = -0.0803
Gene TDP1: Pearson Correlation = -0.0303
Gene AIFM2: Pearson Correlation = -0.0555
Gene SPATA7: Pearson Correlation = 0.0810
Gene MED17: Pearson Correlation = 0.0352
Gene RETSAT: Pearson Correlation = -0.0083
Gene CAPG: Pearson Correlation = -0.0384
Gene AP2S1: Pearson Correlation = -0.0545
Gene USH2A: Pearson Correlation = 0.0552
Gene ZPBP: Pearson Correlation = 0.0374
Gene TG: Pearson Correlation = -0.0017
Gene ADAM28: Pears

/home/macaulay/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/macaulay/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Model 11 loaded successfully for epoch 12
485500
Gene PHF23: Pearson Correlation = 0.0500
Gene CDH10: Pearson Correlation = 0.0192
Gene INPP4A: Pearson Correlation = -0.0673
Gene RAB27B: Pearson Correlation = 0.0604
Gene PSMA4: Pearson Correlation = -0.0066
Gene MYO16: Pearson Correlation = -0.0100
Gene LSG1: Pearson Correlation = 0.0955
Gene PARP3: Pearson Correlation = 0.0140
Gene TNC: Pearson Correlation = -0.0658
Gene THAP3: Pearson Correlation = 0.0710
Gene RIPOR3: Pearson Correlation = -0.0803
Gene TDP1: Pearson Correlation = -0.0303
Gene AIFM2: Pearson Correlation = -0.0555
Gene SPATA7: Pearson Correlation = 0.0810
Gene MED17: Pearson Correlation = 0.0352
Gene RETSAT: Pearson Correlation = -0.0083
Gene CAPG: Pearson Correlation = -0.0384
Gene AP2S1: Pearson Correlation = -0.0545
Gene USH2A: Pearson Correlation = 0.0552
Gene ZPBP: Pearson Correlation = 0.0374
Gene TG: Pearson Correlation = -0.0017
Gene ADAM28: Pearson Correlation = -0.0337
Gene BARX2: Pearson Correlation = 0.0054

/home/macaulay/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/macaulay/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


485500
Gene PHF23: Pearson Correlation = 0.0500
Gene CDH10: Pearson Correlation = 0.0192
Gene INPP4A: Pearson Correlation = -0.0673
Gene RAB27B: Pearson Correlation = 0.0604
Gene PSMA4: Pearson Correlation = -0.0066
Gene MYO16: Pearson Correlation = -0.0100
Gene LSG1: Pearson Correlation = 0.0955
Gene PARP3: Pearson Correlation = 0.0140
Gene TNC: Pearson Correlation = -0.0658
Gene THAP3: Pearson Correlation = 0.0710
Gene RIPOR3: Pearson Correlation = -0.0803
Gene TDP1: Pearson Correlation = -0.0303
Gene AIFM2: Pearson Correlation = -0.0555
Gene SPATA7: Pearson Correlation = 0.0810
Gene MED17: Pearson Correlation = 0.0352
Gene RETSAT: Pearson Correlation = -0.0083
Gene CAPG: Pearson Correlation = -0.0384
Gene AP2S1: Pearson Correlation = -0.0545
Gene USH2A: Pearson Correlation = 0.0552
Gene ZPBP: Pearson Correlation = 0.0374
Gene TG: Pearson Correlation = -0.0017
Gene ADAM28: Pearson Correlation = -0.0337
Gene BARX2: Pearson Correlation = 0.0054
Gene DCUN1D1: Pearson Correlation = 0.009

/home/macaulay/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/macaulay/.local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    m_counter = 0
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        if m_counter == 0:
            print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
            m_counter += 1
    else:
        if m_counter == 0:
            print('No saved model found. Training from scratch.')
            m_counter += 1
    return model


# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)
    # data1 = data1.iloc[:300]

    # display(data1)
    # display(data2)
    # display(df_Y)



    # import time

    # start_epoch = 1
    # end_epoch = 20
    from sklearn.model_selection import train_test_split
    from scipy.stats import pearsonr
    # print(len(data1))
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs

    for epoch in range(start_epoch, end_epoch):
        max_index = (len(data1) // batch_size1) * batch_size1
        # print(max_index)
        # # time.sleep(10000)

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []

        for j in range(0, max_index, batch_size1):
            batched_data1 = data1.iloc[j:j+batch_size1]
            model.train()
            model = load_model(model, epoch, model_save_path)
            train_correlations = []
            train_lossess = []
            test_correlations = []
            test_lossess = []
            for batch_X, batch_Y in cartesian_product_generator(batched_data1, data2, df_Y, batch_size1):
                
                # Concatenate X and Y DataFrames along the columns axis
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)            

                
                # Split  data back into X and Y
                datas_X = combined.iloc[:, :-batch_Y.shape[1]]
                datas_Y = combined.iloc[:, -batch_Y.shape[1]:]

                X_train = datas_X.iloc[:int(len(datas_X)-9710)]
                X_test = datas_X.iloc[int(len(datas_X)-9710):]
                Y_train = datas_Y.iloc[:int(len(datas_Y)-9710)]
                Y_test = datas_Y.iloc[int(len(datas_Y)-9710):]

                # display(X_test)
                # display(Y_test)

                # Copy gene name
                batch_gene_name = X_test['Gene'].iloc[0]
                # print(batch_gene_name)
                X_train = X_train.drop(columns=["Gene"])
                X_test = X_test.drop(columns=["Gene"])

                # print(X_train.select_dtypes(include=['object']).head())
                # display(X_test)
                # # display(Y_train)
                # display(Y_test)
                # import time
                
                # Convert training data to tensor and create dataloader
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations.append(train_correlation)
                    train_lossess.append(loss.item())

                # Evaluate on test data
                model.eval()
                with torch.no_grad():
                    # Convert test data to tensor and evaluate
                    X_test_tensor = torch.tensor(X_test.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    # Convert tensors to numpy arrays for correlation computation
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    epoch_test_correlations.append(correlation)
                    epoch_test_losses.append(test_loss.item())

                    # Aggregate predictions and actual values for each gene
                    for gene in X_test['Gene'].unique():
                        gene_mask = (X_test['Gene'] == gene)
                        X_test_gene = X_test[gene_mask]
                        Y_test_gene = Y_test[gene_mask]

                        X_test_tensor = torch.tensor(X_test_gene.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                        Y_test_tensor = torch.tensor(Y_test_gene.values.reshape(-1, 1), dtype=torch.float32).to(device)

                        predictions_gene = model(X_test_tensor)

                        predictions_gene_np = predictions_gene.cpu().numpy().squeeze()
                        Y_test_gene_np = Y_test_tensor.cpu().numpy().squeeze()
                        gene_predictions[gene] = gene_predictions.get(gene, []) + predictions_gene_np.tolist()
                        gene_actuals[gene] = gene_actuals.get(gene, []) + Y_test_gene_np.tolist()

            # Calculate Pearson correlation for each gene
            gene_correlations = {}
            for gene in gene_predictions.keys():
                correlation, _ = pearsonr(gene_predictions[gene], gene_actuals[gene])
                gene_correlations[gene] = correlation

            # Print or save gene correlations
            for gene, corr in gene_correlations.items():
                print(f"Gene {gene}: Pearson Correlation = {corr:.4f}")
                with open(os.path.join(model_save_path, f'gene_correlations.txt'), 'a') as f:
                    if epoch == 1:
                        f.write('Epoch,gene,correlation\n')
                    f.write(f'Epoch{epoch},{gene},{corr:.4f}\n')
                   

            # Save model state and metrics
            torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
            with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
                if epoch == 1:
                    f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
                f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')




main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=1,learning_rate=0.001,num_epochs=10,
                   model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

   


In [6]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np




def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model


# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)
    # data1 = data1.iloc[:300]

    # display(data1)
    # display(data2)
    # display(df_Y)



    # import time

    # start_epoch = 1
    # end_epoch = 20
    from sklearn.model_selection import train_test_split
    from scipy.stats import pearsonr
    # print(len(data1))
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs


    gene_predictions = {}
    gene_actuals = {}



    for epoch in range(start_epoch, end_epoch):
        max_index = (len(data1) // batch_size1) * batch_size1
        # print(max_index)
        # # time.sleep(10000)

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []
        test_counter = 0
        for j in range(0, max_index, batch_size1):
            batched_data1 = data1.iloc[j:j+batch_size1]
            model.train()
            model = load_model(model, epoch, model_save_path)
            train_correlations = []
            train_lossess = []
            test_correlations = []
            test_lossess = []
            for batch_X, batch_Y in cartesian_product_generator(batched_data1, data2, df_Y, batch_size1):
                
                # Concatenate X and Y DataFrames along the columns axis
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)            

                # Shuffle data
                combined = combined.sample(frac=1, random_state=42).reset_index(drop=True)
                display(combined)
                


                # Split shuffled data back into X and Y
                shuffled_X = combined.iloc[:, :-batch_Y.shape[1]]
                shuffled_Y = combined.iloc[:, -batch_Y.shape[1]:]

                # Then use train_test_split to split the shuffled data into training and test sets
                X_train, X_test, Y_train, Y_test = train_test_split(shuffled_X, shuffled_Y, test_size=0.1, shuffle=False)

                if epoch == 1:

                    X_test.to_csv(model_save_path, f'X_test_{test_counter}.csv', index=False)
                    Y_test.to_csv(model_save_path, f'Y_test_{test_counter}.csv', index=False)
                    X_train = X_train.drop(columns=["Gene"])
                    
                    test_counter += 1
                else:
                    X_test = pd.read_csv(os.path.join(model_save_path, f'X_test_{test_counter}.csv'))
                    Y_test = pd.read_csv(os.path.join(model_save_path, f'Y_test_{test_counter}.csv'))
                    X_train = X_train.drop(columns=["Gene"])
                    
                    test_counter += 1

                # Convert training data to tensor and create dataloader
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations.append(train_correlation)
                    train_lossess.append(loss.item())

                # Evaluate on test data
                model.eval()
                with torch.no_grad():
                    # Convert test data to tensor and evaluate
                    X_test_tensor = torch.tensor(X_test.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    # Convert tensors to numpy arrays for correlation computation
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    epoch_test_correlations.append(correlation)
                    epoch_test_losses.append(test_loss.item())

                    # Aggregate predictions and actual values for each gene
                    for gene in X_test['Gene'].unique():
                        gene_mask = (X_test['Gene'] == gene)
                        X_test_gene = X_test[gene_mask]
                        Y_test_gene = Y_test[gene_mask]

                        X_test_tensor = torch.tensor(X_test_gene.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                        Y_test_tensor = torch.tensor(Y_test_gene.values.reshape(-1, 1), dtype=torch.float32).to(device)

                        predictions_gene = model(X_test_tensor)

                        predictions_gene_np = predictions_gene.cpu().numpy().squeeze()
                        Y_test_gene_np = Y_test_tensor.cpu().numpy().squeeze()
                        gene_predictions[gene] = gene_predictions.get(gene, []) + predictions_gene_np.tolist()
                        gene_actuals[gene] = gene_actuals.get(gene, []) + Y_test_gene_np.tolist()

            # Calculate Pearson correlation for each gene
            gene_correlations = {}
            for gene in gene_predictions.keys():
                correlation, _ = pearsonr(gene_predictions[gene], gene_actuals[gene])
                gene_correlations[gene] = correlation

            # Print or save gene correlations
            for gene, corr in gene_correlations.items():
                print(f"Gene {gene}: Pearson Correlation = {corr:.4f}")
                with open(os.path.join(model_save_path, f'gene_correlations.txt'), 'a') as f:
                    if epoch == 1:
                        f.write('Epoch,gene,correlation\n')
                    f.write(f'Epoch{epoch},{gene},{corr:.4f}\n')
                   

        # Save model state and metrics
        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
            if epoch == 1:
                f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
            f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')




main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=500,learning_rate=0.001,num_epochs=10,
                   model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

   


Device: cuda
Number of available GPUs: 1


Training data loaded successfully
Model initialized successfully
Model 40 loaded successfully for epoch 41
4855


,Gene,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,...,1891_y,1892_y,1893_y,1894_y,1895_y,1896_y,1897_y,1898_y,1899_y,Y
0,GDE1,0.013251,0.258344,0.148767,-0.011984,-0.054923,0.024582,-0.055574,-0.057606,-0.006614,...,0.286353,0.624002,-0.025035,-0.917881,-0.099783,0.187780,0.342172,0.011078,-0.250831,-0.009199
1,LAMP2,0.004373,-0.052026,0.115090,-0.018382,-0.366196,0.019519,-0.149446,-0.032409,-0.004365,...,-0.172742,0.136557,-0.052210,-0.618007,0.540206,0.036533,0.057426,-0.639297,-0.364172,0.302142
2,GDE1,0.013251,0.258344,0.148767,-0.011984,-0.054923,0.024582,-0.055574,-0.057606,-0.006614,...,0.278565,0.324385,-0.063442,-0.964686,0.107228,0.799806,0.099090,0.167747,-0.421059,0.139715
3,LAMP2,0.004373,-0.052026,0.115090,-0.018382,-0.366196,0.019519,-0.149446,-0.032409,-0.004365,...,-0.450451,-0.049049,-0.187901,-0.538388,0.397562,0.206421,-0.045652,-0.187902,-0.764973,0.223350
4,LAMP2,0.004373,-0.052026,0.115090,-0.018382,-0.366196,0.019519,-0.149446,-0.032409,-0.004365,...,0.359132,0.155745,0.042304,-0.320793,-0.003410,0.212727,0.084276,-0.159299,0.060637,0.296481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4850,GDE1,0.013251,0.258344,0.148767,-0.011984,-0.054923,0.024582,-0.055574,-0.057606,-0.006614,...,-0.485712,-0.058002,-0.147350,-0.560812,0.551601,0.331454,-0.106176,-0.510723,-0.617542,0.006007
4851,ZFX,0.013528,-0.073574,0.050425,-0.007168,-0.083504,0.009434,-0.243107,-0.036453,-0.003885,...,0.443331,0.311685,0.045211,-0.834403,-0.088015,0.467807,0.119659,-0.223629,-0.147326,-0.345077
4852,ASB4,0.003966,-0.157932,0.034745,-0.003395,0.023854,-0.018922,0.011001,-0.031456,-0.005197,...,-0.183564,-0.193609,-0.424348,-0.473449,0.642013,-0.181230,0.091139,-0.508000,-0.591336,0.185803
4853,ASB4,0.003966,-0.157932,0.034745,-0.003395,0.023854,-0.018922,0.011001,-0.031456,-0.005197,...,-0.279301,0.160942,-0.108084,-0.244876,0.433558,0.201031,-0.339610,-0.513008,-0.662369,0.094623


KeyboardInterrupt: 

: 

In [9]:
! cd datas/ && rm *


import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np
import json
import sys



def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    # print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')

    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model


# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)
    # data1 = data1.iloc[:300]

    # display(data1)
    # display(data2)
    # display(df_Y)



    # import time

    # start_epoch = 1
    # end_epoch = 20
    from sklearn.model_selection import train_test_split
    from scipy.stats import pearsonr
    # print(len(data1))
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs


    gene_predictions = {}
    gene_actuals = {}



    for epoch in range(start_epoch, end_epoch):
        max_index = (len(data1) // batch_size1) * batch_size1
        # print(max_index)
        # # time.sleep(10000)
        max_index = 50 

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []
        test_counter = 0
        for j in range(0, max_index, batch_size1):
            batched_data1 = data1.iloc[j:j+batch_size1]
            model.train()
            if j == 0:
                model = load_model(model, epoch, model_save_path)

            for batch_X, batch_Y in cartesian_product_generator(batched_data1, data2, df_Y, batch_size1):
                
                # Concatenate X and Y DataFrames along the columns axis
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)            

                # Shuffle data
                combined = combined.sample(frac=1, random_state=42).reset_index(drop=True)
                
            

                # Split shuffled data back into X and Y
                shuffled_X = combined.iloc[:, :-batch_Y.shape[1]]
                shuffled_Y = combined.iloc[:, -batch_Y.shape[1]:]

                # Then use train_test_split to split the shuffled data into training and test sets
                X_train, X_test, Y_train, Y_test = train_test_split(shuffled_X, shuffled_Y, test_size=0.05, shuffle=False)
                # X_test_indices = X_test.index.tolist()  # Get indices of X_test
                # Y_test_indices = Y_test.index.tolist() 


                # if epoch == 1:

                #     sys.exit('stop')
                #     file_name_X = f'X_test_{test_counter}.csv'
                #     file_path_X = os.path.join(model_save_path, file_name_X)
                #     file_name_Y = f'Y_test_{test_counter}.csv'
                #     file_path_Y = os.path.join(model_save_path, file_name_Y)
                #     X_test.to_csv(file_path_X, index=False)
                #     Y_test.to_csv(file_path_Y, index=False)
                #     X_train = X_train.drop(columns=["Gene"])
                    
                #     test_counter += 1
                 
                # else:
                #     sys.exit('stop')
                #     file_name_X = f'X_test_{test_counter}.csv'
                #     file_path_X = os.path.join(model_save_path, file_name_X)
                #     X_test = pd.read_csv(file_path_X)
                #     file_name_Y = f'Y_test_{test_counter}.csv'
                #     file_path_Y = os.path.join(model_save_path, file_name_Y)
                #     Y_test = pd.read_csv(file_path_Y)
                #     X_train = X_train.drop(columns=["Gene"])
                    
                #     test_counter += 1


                # import sys
                # sys.exit('stop')
                # Convert training data to tensor and create dataloader
                X_train = X_train.drop(columns=["Gene"])
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                train_correlations_b = []
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations_b.append(train_correlation)
                epoch_train_losses.append(train_loss / len(train_dataloader))
                epoch_train_correlations.append(np.mean(train_correlations_b))



                # Evaluate on test data
                model.eval()
                with torch.no_grad():
                    # Convert test data to tensor and evaluate
                    X_test_tensor = torch.tensor(X_test.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    # Convert tensors to numpy arrays for correlation computation
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    epoch_test_correlations.append(correlation)
                    epoch_test_losses.append(test_loss.item())

                    # Aggregate predictions and actual values for each gene
                    for gene in X_test['Gene'].unique():
                        gene_mask = (X_test['Gene'] == gene)
                        X_test_gene = X_test[gene_mask]
                        Y_test_gene = Y_test[gene_mask]

                        X_test_tensor = torch.tensor(X_test_gene.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                        Y_test_tensor = torch.tensor(Y_test_gene.values.reshape(-1, 1), dtype=torch.float32).to(device)

                        predictions_gene = model(X_test_tensor)

                        predictions_gene_np = predictions_gene.cpu().numpy().squeeze()
                        Y_test_gene_np = Y_test_tensor.cpu().numpy().squeeze()
                        gene_predictions[gene] = gene_predictions.get(gene, []) + predictions_gene_np.tolist()
                        gene_actuals[gene] = gene_actuals.get(gene, []) + Y_test_gene_np.tolist()

                # Calculate Pearson correlation for each gene
                gene_correlations = {}
                for gene in gene_predictions.keys():
                    correlation, _ = pearsonr(gene_predictions[gene], gene_actuals[gene])
                    gene_correlations[gene] = correlation

        # Print or save gene correlations
        for gene, corr in gene_correlations.items():
            print(f"Gene {gene}: Pearson Correlation = {corr:.4f}")

            with open(os.path.join(model_save_path, f'gene_correlations.txt'), 'a') as f:
                if epoch == 1:
                    f.write('Epoch,gene,correlation\n')
                f.write(f'Epoch{epoch},{gene},{corr:.4f}\n')
                   

        # Save model state and metrics
        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
            if epoch == 1:
                f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
            f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')
        print(f'Epoch {epoch} completed')




main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=1,learning_rate=0.001,num_epochs=6,
                   model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

                #    model_save_path='/mnt/data/macaulay/datas/model_datas/',test_batch_size=128)

# model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

Device: cuda
Number of available GPUs: 1
Training data loaded successfully
Model initialized successfully
No saved model found. Training from scratch.
Gene ZFX: Pearson Correlation = 0.0390
Gene LAMP2: Pearson Correlation = -0.0142
Gene ITGA2B: Pearson Correlation = 0.0706
Gene ASB4: Pearson Correlation = 0.0536
Gene GDE1: Pearson Correlation = -0.1093
Gene REX1BD: Pearson Correlation = -0.1915
Gene CRLF1: Pearson Correlation = 0.1670
Gene OSBPL7: Pearson Correlation = 0.0709
Gene TMEM98: Pearson Correlation = 0.0469
Gene YBX2: Pearson Correlation = 0.0517
Gene KRT33A: Pearson Correlation = 0.0871
Gene ABCC8: Pearson Correlation = 0.0427
Gene CACNG3: Pearson Correlation = 0.0575
Gene TMEM132A: Pearson Correlation = 0.1294
Gene AP2B1: Pearson Correlation = 0.0556
Gene TAC1: Pearson Correlation = 0.0902
Gene ZNF263: Pearson Correlation = 0.0818
Gene CX3CL1: Pearson Correlation = -0.0353
Gene SPATA20: Pearson Correlation = 0.0329
Gene CACNA1G: Pearson Correlation = 0.1588
Gene TNFRSF12A: 

In [ ]:
! cd datas/ && rm *


import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np
import json
import sys



def initialize_environment(data1_path, data2_path, df_Y_path, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')

    data1 = pd.read_csv(data1_path)
    data2 = pd.read_csv(data2_path)
    df_Y = pd.read_csv(df_Y_path)
    print('Training data loaded successfully')

    loss_fn = nn.MSELoss()
    input_dim = data1.shape[1] + data2.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')
    
    
    return device, data1, data2, df_Y, model, optimizer, loss_fn

def cartesian_product(data1, data2):
    data1['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(data1, data2, on='key').drop(columns=['key'])
    # print(len(combined_data))
    
    return combined_data

def cartesian_product_generator(data1, data2, df_Y, batch_size1):
    for i in range(0, len(data1), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = data1.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y

def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')

    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model


# Main Training Loop
def main_training_loop(data1_path, data2_path, df_Y_path, 
                               data1_test_path_A, data2_test_path_A, df_Y_test_path_A, 
                               data1_test_path_B, data2_test_path_B, df_Y_test_path_B,
                               data1_test_path_C, data2_test_path_C, df_Y_test_path_C,
                               batch_size1, learning_rate, num_epochs, model_save_path, test_batch_size=128):
    

    device, data1, data2, df_Y, model, optimizer, loss_fn = initialize_environment(data1_path, data2_path, df_Y_path, learning_rate)
    # data1 = data1.iloc[:300]

    # display(data1)
    # display(data2)
    # display(df_Y)



    # import time

    # start_epoch = 1
    # end_epoch = 20
    from sklearn.model_selection import train_test_split
    from scipy.stats import pearsonr
    # print(len(data1))
    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs


    gene_predictions = {}
    gene_actuals = {}



    for epoch in range(start_epoch, end_epoch):
        max_index = (len(data1) // batch_size1) * batch_size1
        # print(max_index)
        # # time.sleep(10000)
        max_index = 50 

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []
        test_counter = 0
        for j in range(0, max_index, batch_size1):
            batched_data1 = data1.iloc[j:j+batch_size1]
            model.train()
            if j == 0:
                model = load_model(model, epoch, model_save_path)

            for batch_X, batch_Y in cartesian_product_generator(batched_data1, data2, df_Y, batch_size1):
                
                # Concatenate X and Y DataFrames along the columns axis
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)            

                # Shuffle data
                combined = combined.sample(frac=1, random_state=42).reset_index(drop=True)
                
            

                # Split shuffled data back into X and Y
                shuffled_X = combined.iloc[:, :-batch_Y.shape[1]]
                shuffled_Y = combined.iloc[:, -batch_Y.shape[1]:]

                # Then use train_test_split to split the shuffled data into training and test sets
                X_train, X_test, Y_train, Y_test = train_test_split(shuffled_X, shuffled_Y, test_size=0.05, shuffle=False)
                # X_test_indices = X_test.index.tolist()  # Get indices of X_test
                # Y_test_indices = Y_test.index.tolist() 


                # if epoch == 1:

                #     sys.exit('stop')
                #     file_name_X = f'X_test_{test_counter}.csv'
                #     file_path_X = os.path.join(model_save_path, file_name_X)
                #     file_name_Y = f'Y_test_{test_counter}.csv'
                #     file_path_Y = os.path.join(model_save_path, file_name_Y)
                #     X_test.to_csv(file_path_X, index=False)
                #     Y_test.to_csv(file_path_Y, index=False)
                #     X_train = X_train.drop(columns=["Gene"])
                    
                #     test_counter += 1
                 
                # else:
                #     sys.exit('stop')
                #     file_name_X = f'X_test_{test_counter}.csv'
                #     file_path_X = os.path.join(model_save_path, file_name_X)
                #     X_test = pd.read_csv(file_path_X)
                #     file_name_Y = f'Y_test_{test_counter}.csv'
                #     file_path_Y = os.path.join(model_save_path, file_name_Y)
                #     Y_test = pd.read_csv(file_path_Y)
                #     X_train = X_train.drop(columns=["Gene"])
                    
                #     test_counter += 1


                # import sys
                # sys.exit('stop')
                # Convert training data to tensor and create dataloader
                X_train = X_train.drop(columns=["Gene"])
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                train_correlations_b = []
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations_b.append(train_correlation)
                epoch_train_losses.append(train_loss / len(train_dataloader))
                epoch_train_correlations.append(np.mean(train_correlations_b))



                # Evaluate on test data
                model.eval()
                with torch.no_grad():
                    # Convert test data to tensor and evaluate
                    X_test_tensor = torch.tensor(X_test.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    # Convert tensors to numpy arrays for correlation computation
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    epoch_test_correlations.append(correlation)
                    epoch_test_losses.append(test_loss.item())

                    # Aggregate predictions and actual values for each gene
                    for gene in X_test['Gene'].unique():
                        gene_mask = (X_test['Gene'] == gene)
                        X_test_gene = X_test[gene_mask]
                        Y_test_gene = Y_test[gene_mask]

                        X_test_tensor = torch.tensor(X_test_gene.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                        Y_test_tensor = torch.tensor(Y_test_gene.values.reshape(-1, 1), dtype=torch.float32).to(device)

                        predictions_gene = model(X_test_tensor)

                        predictions_gene_np = predictions_gene.cpu().numpy().squeeze()
                        Y_test_gene_np = Y_test_tensor.cpu().numpy().squeeze()
                        gene_predictions[gene] = gene_predictions.get(gene, []) + predictions_gene_np.tolist()
                        gene_actuals[gene] = gene_actuals.get(gene, []) + Y_test_gene_np.tolist()

                # Calculate Pearson correlation for each gene
                gene_correlations = {}
                for gene in gene_predictions.keys():
                    correlation, _ = pearsonr(gene_predictions[gene], gene_actuals[gene])
                    gene_correlations[gene] = correlation

        # Print or save gene correlations
        for gene, corr in gene_correlations.items():
            print(f"Gene {gene}: Pearson Correlation = {corr:.4f}")

            with open(os.path.join(model_save_path, f'gene_correlations.txt'), 'a') as f:
                if epoch == 1:
                    f.write('Epoch,gene,correlation\n')
                f.write(f'Epoch{epoch},{gene},{corr:.4f}\n')
                   

        # Save model state and metrics
        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
            if epoch == 1:
                f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
            f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')
        print(f'Epoch {epoch} completed')




main_training_loop(data1_path="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_path="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_path='/mnt/data/macaulay/datas/training_crispr.csv',
                   data1_test_path_A="/mnt/data/macaulay/datas/training_gene_embeddings.csv",
                   data2_test_path_A="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_A='/mnt/data/macaulay/datas/A_test_gene__Y_crispr.csv',
                   data1_test_path_B="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_B="/mnt/data/macaulay/datas/training_omicExpression_Embeddings.csv",
                   df_Y_test_path_B='/mnt/data/macaulay/datas/B_test_gene__Y_crispr.csv',
                   data1_test_path_C="/mnt/data/macaulay/datas/test_gene_embeddings.csv",
                   data2_test_path_C="/mnt/data/macaulay/datas/test_omicExpression_Embeddings.csv",
                   df_Y_test_path_C='/mnt/data/macaulay/datas/C_test_gene__Y_crispr.csv',
                   batch_size1=1,learning_rate=0.001,num_epochs=6,
                   model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

                #    model_save_path='/mnt/data/macaulay/datas/model_datas/',test_batch_size=128)

# model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import os
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
from functions import FullConnectedBlock, NeuralNetwork
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
pd.options.mode.chained_assignment = None
import numpy as np
import json
import sys
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr


def initialize_environment(unirep_train, df_omic_train, learning_rate):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print('Device:', device)
    print(f'Number of available GPUs: {torch.cuda.device_count()}')
    loss_fn = nn.MSELoss()
    input_dim = unirep_train.shape[1] + df_omic_train.shape[1] - 1
    model = NeuralNetwork(input_dim)
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    if torch.cuda.device_count() >= 4:
        print("Using 4 GPUs!")
        model = nn.DataParallel(model, device_ids=list(range(4)))
    print('Model initialized successfully')

    return device, model, optimizer, loss_fn


def cartesian_product(unirep_train, data2):
    unirep_train['key'] = 1
    data2['key'] = 1
    combined_data = pd.merge(unirep_train, data2, on='key').drop(columns=['key'])
    return combined_data


def cartesian_product_generator(unirep_train, data2, df_Y, batch_size1):
    for i in range(0, len(unirep_train), batch_size1):
        start_idx = i * len(data2)
        end_idx = (i + batch_size1) * len(data2)
        batch_data1 = unirep_train.iloc[i:i + batch_size1]
        
        batch_X = cartesian_product(batch_data1, data2)
        batch_Y = df_Y.iloc[start_idx:end_idx]
        yield batch_X, batch_Y


def load_model(model, epoch, model_save_path):
    model_path = os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch-1}.pth')

    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f'Model {epoch - 1} loaded successfully for epoch {epoch}')
    else:
        print('No saved model found. Training from scratch.')
    return model


def preprocess(unirep, vae, essentiality):
    df_unirep = pd.read_csv(unirep)
    df_essentiality = pd.read_csv(essentiality)
    df_omic = pd.read_csv(vae)
    
    essentiality_train = df_essentiality.iloc[30:, 101:]
    essentiality_test_A = df_essentiality.iloc[:30, 101:]
    essentiality_test_B = df_essentiality.iloc[30:, 1:101]
    essentiality_test_C = df_essentiality.iloc[:30, 1:101]

    flattened_essentiality_train = flatten_to_1D(essentiality_train, type='column')
    flattened_essentiality_test_A = flatten_to_1D(essentiality_test_A, type='column')
    flattened_essentiality_test_B = flatten_to_1D(essentiality_test_B, type='column')
    flattened_essentiality_test_C = flatten_to_1D(essentiality_test_C, type='column')

    print(f'Length of flattened_essentiality_train: {len(flattened_essentiality_train)}')
    print(f'Length of flattened_essentiality_test_A: {len(flattened_essentiality_test_A)}')
    print(f'Length of flattened_essentiality_test_B: {len(flattened_essentiality_test_B)}')
    print(f'Length of flattened_essentiality_test_C: {len(flattened_essentiality_test_C)}')

    df_omic_train = df_omic.iloc[30:, 1:] #train_omic
    df_omic_test = df_omic.iloc[:30, 1:] #test_omic

    unirep_test = df_unirep.iloc[:100] #test_unirep
    unirep_train = df_unirep.iloc[100:] #train_unirep

    return df_omic_train, df_omic_test, unirep_train, unirep_test, flattened_essentiality_train, flattened_essentiality_test_A, flattened_essentiality_test_B, flattened_essentiality_test_C


def flatten_to_1D(df, type='column'):
    if type == 'column':
        df = df.T
    elif type == 'row':
        df = df

    df = df.values.flatten()
    df = pd.DataFrame(df, columns=['Y'])   
    return df


def main_training_loop(unirep_path, vae_path, essentiality_path,
                        batch_size1, learning_rate, num_epochs,
                          model_save_path, test_batch_size=128):
    
    df_omic_train, df_omic_test, unirep_train, unirep_test, flattened_essentiality_train, flattened_essentiality_test_A, flattened_essentiality_test_B, flattened_essentiality_test_C = preprocess(unirep_path, vae_path, essentiality_path)
    device, model, optimizer, loss_fn = initialize_environment(unirep_train, df_omic_train, learning_rate)

    saved_models = os.listdir(model_save_path)
    epochs = [int(file.split('_')[-1].split('.')[0]) for file in saved_models if 'crispr_fc1_model_state_epoch_' in file]
    last_epoch = max(epochs) if epochs else 0
    start_epoch = last_epoch + 1
    end_epoch = start_epoch + num_epochs

    gene_predictions = {}
    gene_actuals = {}

    for epoch in range(start_epoch, end_epoch):
        max_index = (len(unirep_train) // batch_size1) * batch_size1
        max_index = 500

        epoch_train_correlations = []
        epoch_train_losses = []
        epoch_test_correlations = []
        epoch_test_losses = []
        test_counter = 0
        for j in range(0, max_index, batch_size1):
            batched_data1 = unirep_train.iloc[j:j+batch_size1]
            model.train()
            if j == 0:
                model = load_model(model, epoch, model_save_path)

            for batch_X, batch_Y in cartesian_product_generator(batched_data1, df_omic_train, flattened_essentiality_train, batch_size1):
                
                combined = pd.concat([batch_X.reset_index(drop=True), batch_Y.reset_index(drop=True)], axis=1)           

                combined = combined.sample(frac=1, random_state=42).reset_index(drop=True)
                
                shuffled_X = combined.iloc[:, :-batch_Y.shape[1]]
                shuffled_Y = combined.iloc[:, -batch_Y.shape[1]:]

                X_train, X_test, Y_train, Y_test = train_test_split(shuffled_X, shuffled_Y, test_size=0.05, shuffle=False)
                X_train = X_train.drop(columns=["Gene"])
                X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
                Y_train_tensor = torch.tensor(Y_train.values.reshape(-1, 1), dtype=torch.float32).to(device)
                train_data = TensorDataset(X_train_tensor, Y_train_tensor)
                train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)
                
                train_loss = 0.0
                train_correlations_b = []
                for inputs, targets in train_dataloader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = loss_fn(outputs, targets)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    train_predictions_np = outputs.detach().cpu().numpy()
                    Y_train_np = targets.cpu().numpy()
                    train_correlation, _ = pearsonr(train_predictions_np.squeeze(), Y_train_np.squeeze())
                    train_correlations_b.append(train_correlation)
                epoch_train_losses.append(train_loss / len(train_dataloader))
                epoch_train_correlations.append(np.mean(train_correlations_b))


                model.eval()
                with torch.no_grad():
                    X_test_tensor = torch.tensor(X_test.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                    Y_test_tensor = torch.tensor(Y_test.values.reshape(-1, 1), dtype=torch.float32).to(device)
                    predictions = model(X_test_tensor)
                    test_loss = loss_fn(predictions, Y_test_tensor)
                    
                    predictions_np = predictions.cpu().numpy()
                    Y_test_np = Y_test_tensor.cpu().numpy()
                    
                    correlation, _ = pearsonr(predictions_np.squeeze(), Y_test_np.squeeze())
                    epoch_test_correlations.append(correlation)
                    epoch_test_losses.append(test_loss.item())

                    for gene in X_test['Gene'].unique():
                        gene_mask = (X_test['Gene'] == gene)
                        X_test_gene = X_test[gene_mask]
                        Y_test_gene = Y_test[gene_mask]

                        X_test_tensor = torch.tensor(X_test_gene.drop(columns=["Gene"]).values, dtype=torch.float32).to(device)
                        Y_test_tensor = torch.tensor(Y_test_gene.values.reshape(-1, 1), dtype=torch.float32).to(device)

                        predictions_gene = model(X_test_tensor)

                        predictions_gene_np = predictions_gene.cpu().numpy().squeeze()
                        Y_test_gene_np = Y_test_tensor.cpu().numpy().squeeze()
                        gene_predictions[gene] = gene_predictions.get(gene, []) + predictions_gene_np.tolist()
                        gene_actuals[gene] = gene_actuals.get(gene, []) + Y_test_gene_np.tolist()

                gene_correlations = {}
                for gene in gene_predictions.keys():
                    correlation, _ = pearsonr(gene_predictions[gene], gene_actuals[gene])
                    gene_correlations[gene] = correlation

        for gene, corr in gene_correlations.items():
            print(f"Gene {gene}: Pearson Correlation = {corr:.4f}")

            with open(os.path.join(model_save_path, f'gene_correlations.txt'), 'a') as f:
                if epoch == 1:
                    f.write('Epoch,gene,correlation\n')
                f.write(f'Epoch{epoch},{gene},{corr:.4f}\n')
                   

        torch.save(model.state_dict(), os.path.join(model_save_path, f'crispr_fc1_model_state_epoch_{epoch}.pth'))
        with open(os.path.join(model_save_path, f'metrics.txt'), 'a') as f:
            if epoch == 1:
                f.write('epoch,train_loss,train_correlation,test_loss,test_correlation\n')
            f.write(f'{epoch},{np.mean(epoch_train_losses):.4f},{np.mean(epoch_train_correlations):.4f},{np.mean(epoch_test_losses):.4f},{np.mean(epoch_test_correlations):.4f}\n')
        print(f'Epoch {epoch} completed')


main_training_loop(unirep_path="/mnt/data/macaulay/datas/processed_gene_embeddings.csv",
                   vae_path="/mnt/data/macaulay/datas/OmicExpression_embeddings/OmicExpression_embeddings_15.csv",
                   essentiality_path='/mnt/data/macaulay/datas/processed_CRISPRGeneEffect.csv',
                   batch_size1=1,learning_rate=0.001,num_epochs=12,
                   model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)

# model_save_path='/home/macaulay/macaulay/GenePedia/datas/',test_batch_size=128)"


Length of flattened_essentiality_train: 17034253
Length of flattened_essentiality_test_A: 526290
Length of flattened_essentiality_test_B: 97100
Length of flattened_essentiality_test_C: 3000
Device: cuda
Number of available GPUs: 1
Model initialized successfully
Model 12 loaded successfully for epoch 13
Gene ZFX: Pearson Correlation = 0.2568
Gene LAMP2: Pearson Correlation = 0.2464
Gene ITGA2B: Pearson Correlation = 0.2177
Gene ASB4: Pearson Correlation = 0.2271
Gene GDE1: Pearson Correlation = 0.1865
Gene REX1BD: Pearson Correlation = 0.2863
Gene CRLF1: Pearson Correlation = 0.1900
Gene OSBPL7: Pearson Correlation = 0.2670
Gene TMEM98: Pearson Correlation = 0.2208
Gene YBX2: Pearson Correlation = 0.2691
Gene KRT33A: Pearson Correlation = 0.2188
Gene ABCC8: Pearson Correlation = 0.2253
Gene CACNG3: Pearson Correlation = 0.2647
Gene TMEM132A: Pearson Correlation = 0.2591
Gene AP2B1: Pearson Correlation = 0.2372
Gene TAC1: Pearson Correlation = 0.2385
Gene ZNF263: Pearson Correlation = 0.

KeyboardInterrupt: 